# Marker Gene Analysis


Input: kallisto-bustools with multimapping, data subset to 6000 highly variable genes before scvi for batch correction

In [1]:
# Setup

import anndata
from anndata import AnnData
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scanpy as sc

# matplotlib settings
# plt.rcParams.keys() for options

fsize=20
plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

# scanpy settings

sc.set_figure_params(dpi_save = 400, fontsize = 20)

## Load adata

In [ ]:
# Load AnnData

adata = anndata.read_h5ad("/Volumes/Mac-External/insulator/results/insulator_withoutcp190_mito5_multimap_6000hvg_070521.h5ad")

In [ ]:
adata

In [ ]:
# remove mitochondrial and ribosomal genes
riboS = adata.var["gene"].str.startswith('RpS')
riboS = riboS.astype(bool)

riboL = adata.var["gene"].str.startswith('RpL')
riboL = riboL.astype(bool)

mriboS = adata.var["gene"].str.startswith('mRpS')
mriboS = mriboS.astype(bool)

mriboL = adata.var["gene"].str.startswith('mRpL')
mriboL = mriboL.astype(bool)

mito_genes = adata.var["gene"].str.startswith('mt:')
mito_genes = mito_genes.astype(bool)

badgenes = (riboS | riboL | mito_genes | riboS | riboL)

adata = adata[:, ~badgenes]

In [ ]:
adata

## Marker Gene Analysis

In [ ]:
# rank genes for characterizing groups, expects log data
sc.tl.rank_genes_groups(adata, 'leiden_scVI', method='wilcoxon', corr_method="bonferroni", use_raw = False, layer = "norm_log_unscaled")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# save top 10 per cluster to csv file
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10).to_csv('top10markers.csv', index = False, sep = '\t')

In [ ]:
# hierarchical clustering of clusby leiden
sc.tl.dendrogram(adata, groupby = "leiden_scVI")

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata, 
                                n_genes=4, 
                                use_raw=False, 
                                swap_axes=True, 
                                vmin=-3, vmax=3, 
                                cmap='RdBu_r', 
                                figsize=(10,15),
                                save = '1_markergeneheatmap.png')